In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import uuid

# 定义MKP实例
K = 3  # 背包数量
L = 4  # 物品数量
v = np.array([10, 20, 15, 25])  # 物品价值
w = np.array([5, 10, 8, 12])   # 物品重量
W = np.array([15, 20, 18])     # 背包容量
n_qubits = K * L  # 量子比特数 (x_ij变量)

# 惩罚系数
lambda_1 = 50  # 指数项惩罚系数
lambda_2 = 3   # 指数中的缩放因子
lambda_ub = K * np.sum(v)  # 惩罚上界

# 目标函数：-sum(v_j * x_ij)
def objective_function(x):
    x_matrix = x.reshape(K, L)
    return -np.sum(v * x_matrix)

# 约束：h_i(x) = sum(w_j * x_ij) - W_i <= 0
#       h_j(x) = sum(x_ij) - 1 <= 0
def constraint_functions(x):
    x_matrix = x.reshape(K, L)
    h1 = np.sum(w * x_matrix, axis=1) - W  # 背包容量约束
    h2 = np.sum(x_matrix, axis=0) - 1     # 物品分配约束
    return np.concatenate([h1, h2])

# 指数惩罚：sum(lambda_1 * exp(lambda_2 * h_i(x)))
def exponential_penalty(x):
    h = constraint_functions(x)
    return lambda_1 * np.sum(np.exp(lambda_2 * h))

# 构建目标哈密顿量：-sum(v_j * x_ij)
def construct_objective_hamiltonian():
    terms = []
    for i in range(K):
        for j in range(L):
            idx = i * L + j
            coeff = -v[j] / 2
            pauli_str = ['I'] * n_qubits
            pauli_str[idx] = 'Z'
            terms.append((coeff, ''.join(pauli_str)))
            terms.append((-v[j] / 2, 'I' * n_qubits))  # 常数项
    return SparsePauliOp.from_list(terms)

# 构建约束哈密顿量：h_i(x) = sum(w_j * x_ij) - W_i 等
def construct_constraint_hamiltonians():
    constraint_hamiltonians = []
    # 背包约束
    for i in range(K):
        terms = []
        constant = -W[i]
        for j in range(L):
            idx = i * L + j
            coeff = w[j] / 2
            pauli_str = ['I'] * n_qubits
            pauli_str[idx] = 'Z'
            terms.append((coeff, ''.join(pauli_str)))
            constant += w[j] / 2
        terms.append((constant, 'I' * n_qubits))
        constraint_hamiltonians.append(SparsePauliOp.from_list(terms))
    # 物品约束
    for j in range(L):
        terms = []
        constant = -1
        for i in range(K):
            idx = i * L + j
            coeff = 1 / 2
            pauli_str = ['I'] * n_qubits
            pauli_str[idx] = 'Z'
            terms.append((coeff, ''.join(pauli_str)))
            constant += 1 / 2
        terms.append((constant, 'I' * n_qubits))
        constraint_hamiltonians.append(SparsePauliOp.from_list(terms))
    return constraint_hamiltonians

# 计算指数惩罚的期望值（经典计算）
def compute_exponential_penalty_expectation(counts, constraint_hamiltonians):
    total_penalty = 0
    for hamiltonian in constraint_hamiltonians:
        # 计算H_i的本征值
        eigenvalues = []
        for state in range(2 ** n_qubits):
            binary = format(state, f'0{n_qubits}b')
            eigval = 0
            for pauli, coeff in hamiltonian:
                if pauli.to_label() == 'I' * n_qubits:
                    eigval += coeff
                else:
                    idx = pauli.to_label().index('Z')
                    bit = int(binary[idx])
                    eigval += coeff * (1 if bit == 0 else -1)
            eigenvalues.append(eigval)
        # 计算<exp(lambda_2 * H_i)>
        penalty = 0
        for state, count in counts.items():
            idx = int(state, 2)
            prob = count / sum(counts.values())
            penalty += prob * np.exp(lambda_2 * eigenvalues[idx])
        total_penalty += lambda_1 * penalty
    return total_penalty

# QAOA成本函数
def qaoa_cost(params, hamiltonian, constraint_hamiltonians):
    p = 1  # QAOA层数
    circuit = QuantumCircuit(n_qubits)
    # 初始态：|+>^n
    circuit.h(range(n_qubits))
    # 应用QAOA层
    gamma = params[:p]
    beta = params[p:]
    for i in range(p):
        # 问题哈密顿量阶段（RZ和RZZ门）
        for pauli, coeff in hamiltonian:
            if pauli.to_label() != 'I' * n_qubits:
                idx = pauli.to_label().index('Z')
                circuit.rz(2 * gamma[i] * coeff, idx)
        # 混合哈密顿量（RX门）
        for j in range(n_qubits):
            circuit.rx(2 * beta[i], j)
    # 运行电路获取概率分布
    backend = Aer.get_backend('statevector_simulator')
    circuit.measure_all()
    result = backend.run(circuit, shots=1000).result()
    counts = result.get_counts()
    # 计算期望值
    objective_exp = 0
    for pauli, coeff in hamiltonian:
        exp_val = 0
        for state, count in counts.items():
            prob = count / sum(counts.values())
            if pauli.to_label() == 'I' * n_qubits:
                exp_val += coeff * prob
            else:
                idx = pauli.to_label().index('Z')
                bit = int(state[idx])
                exp_val += coeff * (1 if bit == 0 else -1) * prob
        objective_exp += exp_val
    penalty_exp = compute_exponential_penalty_expectation(counts, constraint_hamiltonians)
    return objective_exp + penalty_exp

# 主程序
hamiltonian = construct_objective_hamiltonian()
constraint_hamiltonians = construct_constraint_hamiltonians()
optimizer = minimize
initial_params = np.random.uniform(0, np.pi, 2)  # gamma和beta for p=1
result = optimizer(lambda params: qaoa_cost(params, hamiltonian, constraint_hamiltonians),
                   initial_params, method='L-BFGS-B', options={'maxiter': 100})
optimal_params = result.x

# 评估最终解
circuit = QuantumCircuit(n_qubits)
circuit.h(range(n_qubits))
for i in range(1):
    for pauli, coeff in hamiltonian:
        if pauli.to_label() != 'I' * n_qubits:
            idx = pauli.to_label().index('Z')
            circuit.rz(2 * optimal_params[i] * coeff, idx)
    for j in range(n_qubits):
        circuit.rx(2 * optimal_params[i + 1], j)
circuit.measure_all()
backend = Aer.get_backend('statevector_simulator')
result = backend.run(circuit, shots=1000).result()
counts = result.get_counts()
best_solution = max(counts, key=counts.get)
x_opt = np.array([int(bit) for bit in best_solution])
print(f"最优解: {x_opt.reshape(K, L)}")
print(f"目标值: {objective_function(x_opt)}")
print(f"约束满足: {all(constraint_functions(x_opt) <= 0)}")

TypeError: object of type 'numpy.float64' has no len()